# Clase 12

- Fecha: 22/11/2022 (**última clase**)
- Temas:
    - NDVI pendiente
    - Geopandas
    - Entornos virtuales y Libs de interés
    - ¿cómo seguir?

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

import json
import plotly.graph_objects as go
import plotly

In [ ]:
plotly.__version__

## NDVI 

In [ ]:
import rasterio as rio
from rasterio.plot import show 

In [ ]:
# ruta de los archivos
path_img_b4 = "datos/geo/landsat/LC08_L1TP_227081_20191012_20191018_01_T1/LC08_L1TP_227081_20191012_20191018_01_T1_B4.TIF"
path_img_b5 = "datos/geo/landsat/LC08_L1TP_227081_20191012_20191018_01_T1/LC08_L1TP_227081_20191012_20191018_01_T1_B5.TIF"

# leo imagen
img_b4 = rio.open(path_img_b4)
img_b5 = rio.open(path_img_b5)

# leo contenido raster
red = img_b4.read(1).astype('float64')
nir = img_b5.read(1).astype('float64')

In [ ]:
np.geterr()

In [ ]:
# >>>>  le decimos que ignore la division por cero <<<<
np.seterr(divide = "ignore", invalid="ignore")

ndvi = (nir - red)/(red + nir)

In [ ]:
# filtramos por si acaso

ndvi[ndvi > 1] = np.nan
ndvi[ndvi < -1] = np.nan

show(ndvi, cmap='Greens')

### Problemas Desafío: otros índices
- Enhanced Vegetation Index (EVI)
- Soil Adjusted Vegetation Index (SAVI)
- Green Chlorophyll Index (GCI)
- Atmospherically Resistant Vegetation Index(ARVI)

## Geopandas continuación

In [ ]:
muni = gpd.read_file("datos/geo/municipio.zip")

muni

In [ ]:
muni.crs

In [ ]:
area = muni.area

area

In [ ]:
muni = muni.to_crs(epsg=32633)



In [ ]:
muni["area"] = muni.geometry.area/(10**6)    # explicar sobre columna geometry

#muni.to_crs({'proj':'cea'}) # cilindrical equal-area
#muni["area2"] = muni.area/(10**6)

muni

In [ ]:
muni.explore("area", legend=False)

In [ ]:
#muni = muni.to_crs("WGS84")
#fig = go.Figure()

#plot_muni = go.Choroplethmapbox(geojson=json.loads(muni.to_json()), 
#                                    locations=muni.index, z=muni['area2'],
#                                     marker_line_width=.5)
#fig.add_trace(plot_muni)
#fig.update_layout(mapbox_style="open-street-map", mapbox=dict(center=dict(lat=-31.663, lon=-60.725),zoom=5),)

In [ ]:
idx = muni["nam"]=="Santa Fe"
sfe = muni[idx]
sfe.explore()   # o sfe.plot() me muestra con un gráfico matplotlib 

In [ ]:
muni.centroid

In [ ]:
muni.explore("area", legend=False)

### Pandas dataframe -> Geopandas dataframe

- A partir de un dataframe generamos un geodataframe

In [ ]:
df = pd.DataFrame(
    {'City': ['Buenos Aires', 'Brasilia', 'Santiago', 'Bogota', 'Caracas', 'Santa Fe'],
        'Country': ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Venezuela', 'Argentina'],
        'Latitude': [-34.58, -15.78, -33.45, 4.60, 10.48, -31.6629377],
        'Longitude': [-58.66, -47.91, -70.66, -74.08, -66.86, -60.7252908]})
df

In [ ]:
# convierto los datos de long y lat a POINTS
geom = gpd.points_from_xy(df.Longitude, df.Latitude) # paso las columnas de mi DF

# tengo que definir un sistema de referencia de coordenadas
crs = "EPSG:4326"

In [ ]:
geom

In [ ]:
# ahora que tengo todo armo el geodataframe
gdf = gpd.GeoDataFrame(df, crs = crs, geometry = geom)

gdf

In [ ]:
gdf = gdf.rename_geometry('Puntos')
gdf

In [ ]:
gdf.geometry

In [ ]:
gdf.explore()

In [ ]:
#gdf = gdf.to_crs(epsg=32633)
gdf

In [ ]:
gdf.crs

In [ ]:
coloncito = gdf['Puntos'].iloc[5]
type(coloncito)

In [ ]:
gdf['distancia'] = gdf['Puntos'].distance(coloncito)
gdf

In [ ]:
gdf.to_file("datos/geo/my_file.geojson", driver="GeoJSON")

### Crear geometrías propias



In [ ]:
from shapely.geometry import Point, Polygon

from shapely import wkt

In [ ]:
# acá usamos notación WKT para crear geodataframe
df2 = pd.DataFrame(
    {'City': ['Buenos Aires', 'Brasilia', 'Santiago', 'Bogota', 'Caracas'],
     'Country': ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Venezuela'],
     'Coordinates': ['POINT(-58.66 -34.58)', 'POINT(-47.91 -15.78)',
                     'POINT(-70.66 -33.45)', 'POINT(-74.08 4.60)',
                     'POINT(-66.86 10.48)']})
df2.info()

In [ ]:
df2['Coordinates'] = gpd.GeoSeries.from_wkt(df2['Coordinates'])
df2.info()

In [ ]:
gdf2 = gpd.GeoDataFrame(df2, geometry='Coordinates')

print(gdf2.head())

In [ ]:
gdf2["Coordinates"].x     # es lo mismo que gdf2.geometry.x

In [ ]:
gdf2["Coordinates"].y

**Un polígono**

In [ ]:
# creo una lista de listas con pares lon, lat

lon_lat = [[-60.671750, -31.639597],
           [-60.671749, -31.639655],
           [-60.671663, -31.639654],
           [-60.671654, -31.639596]]

data = pd.DataFrame({"Lugar": "CENEHA", "Aula":"auditorio", "Descripción": "clases Maestria TIG"}, index=[0])

data

In [ ]:
polygon_geom = Polygon(lon_lat)

gdf2 = gpd.GeoDataFrame(data, crs='epsg:4326', geometry=[polygon_geom])     
gdf2

In [ ]:
gdf2.explore()

In [ ]:
data_dic = {'col1': ['name1', 'name2'], 'geometry': [Point(1, 2), Point(2, 1)]}


df3 = gpd.GeoDataFrame(data_dic, crs="EPSG:4326")
df3

## Entornos virtuales

- https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html

**Recomendación:** 
- Tener UN entorno virtual por cada proyecto 
- en directorio de ese proyecto (donde están mis programas) un `requirements.txt` con los paquetes y versiones necesarias
- en ese directorio un README.md con ayuda de cómo crear el entorno

Para no tener que instalar jupyter en cada entorno:

```
$ conda activate tig
(tig)$ conda install ipykernel
(tig)$ ipython kernel install --user --name=tig
(tig)$ conda deactivate
```

## Otras libs
- Streamlit: https://medium.com/spatial-data-science/3-super-helpful-geospatial-web-applications-with-streamlit-d6eae05e2810
- Geopy: https://geopy.readthedocs.io/en/stable/#
- PyOSM: https://pyrosm.readthedocs.io/en/latest/
- Pygis: https://pygis.io/docs/d_access_osm.html
- GeoWombat: https://geowombat.readthedocs.io/en/latest/
- OSMNx: https://www.openstreetmap.org/search?query=santa%20fe%2C%20santa%20fe%2C%20argentina#map=13/-31.6153/-60.7099
- Request: https://requests.readthedocs.io/en/latest/
- Statsmodel: https://www.statsmodels.org/dev/index.html
- Telegram bots: https://python-telegram-bot.org/ (Sofi tutorial: https://www.youtube.com/watch?v=9x1oF5cCd8k)
- GEEmap: https://geemap.org/

### Veamos ejemplo de OSMNx

In [ ]:
sfe = "Santa Fe Capital, Departamento La Capital, Santa Fe, S3000, Argentina"

In [ ]:
import osmnx as ox

tags = {'building': True} # List key-value pairs for tags   

edificios = ox.geometries_from_place(sfe, tags)
edificios.head()


In [ ]:
edificios.explore()

In [ ]:
area

In [ ]:
area.explore()

## FIN

Algunas recomendaciones para seguir:

- **Programar para uno mismo:** resolver problemas repetitivos
- **Buscar un proyecto/problema y enfocarse:** no se puede aprender todo
- **Mantenerse en contacto con comunidades de programación:** lxs nerds suelen ayudar
- **Hacer una tesis que incluya programación:** el resultado de nuestro trabajo puede ser un software que sirva a otros 